In [ ]:
! pip install jupyter-dash
! pip install morfeusz2

In [ ]:
# Imports
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import pandas as pd
import re
import string
import morfeusz2
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import load
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
# def strip_punctuation(text):
#     return text.translate(str.maketrans('', '', string.punctuation))

def strip_non_alphanumeric(text):
    reg = re.compile('[^a-zA-ZĄąĆćĘęŁłŃńÓóŚśŹźŻż]')
    return reg.sub(' ', text)

def replace_whitespace(text):
    reg = re.compile('\s+')
    return reg.sub(' ', text)

def delete_escape_chars(text):
    return text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\r', ' ')

def delete_stop_words(text):
    stop_words_txt = open('drive/MyDrive/Fake News Detection/data/polish.stopwords.txt')
    stop_words = stop_words_txt.read().split('\n')
    stop_words_txt.close()
    return [word for word in text if word not in stop_words]

def preprocess(df):
    # df['Text'] = df['Text'].apply(strip_punctuation)
    df['Text'] = df['Text'].apply(strip_non_alphanumeric)
    df['Text'] = df['Text'].apply(replace_whitespace)
    df['Text'] = df['Text'].apply(delete_escape_chars)
    df['Text'] = df['Text'].apply(str.lower)
    df['Text'] = df['Text'].apply(str.split)
    df['Text'] = df['Text'].apply(delete_stop_words)
    return df

def lemmatise(df):
    morf = morfeusz2.Morfeusz()
    for index, row in df.iterrows():
        text = row['Text']
        lemm_words = []
        for word in text:
            _, _, interpretation = morf.analyse(word)[0]
            lem_word = interpretation[1]
            lem_word_stripped = lem_word.split(':', 1)[0].lower()
            lemm_words.append(lem_word_stripped)
        df.loc[index, 'Text'] = ' '.join(lemm_words)
    return df

In [ ]:
example = '''
Najnowsze dane z raportu brytyjskiej Agencji Bezpieczeństwa Zdrowia 
(UKHSA) jak i Public Health Scotland, instytucji nadzorujących szczepienia 
przeciwko COVID-19 pokazują, że stosunek umieralności z powodu koronawirusa z 
Wuhan jest, jak jedna osoba „niezaszczepiona” do prawie pięć osób „w pełni 
zaszczepionych”. Z podanych informacji widać, że zdecydowana większość 
zgłoszonych zgonów ​​ma miejsce po otrzymaniu zastrzyków. Te informacje są 
wyraźnie sprzeczne z podawanymi przez producentów szczepionek, o ich rzekomej 
ich skuteczności i 90% wskaźniku zapobiegania śmiertelności, jak i 
rozpowszechnianą propagandą przez media i rządy poszczególnych państw. W związku 
z zaistniałą sytuacją angielskie media The Exposé zadały pytanie: „Jeśli 
zastrzyk Covid ma być w 90% skuteczny w zapobieganiu śmierci, dlaczego 
„zaszczepieni” ludzie obecnie umierają w stosunku 4,8:1 do „nieszczepionych” 
osób?”

Wniosek – UKHSA kłamie publicznie twierdząc, że szczepionki ratują życie, co 
jest sprzeczne z tym, co mówią podawane przez nich dane. Kłamią również inne 
instytucje, media jak i rządy promujące ten specyfik. Korzystając z danych VSR 
(Vaccine Surveillance Report) zebranych przez okres trzech miesięcy, The Exposé 
odkrył, że nie tylko odporność ludzi po zaszczepieniu na wrażego chińskiego 
wirusa spadła, ale również wyjątkowo szybko zanika odporność, która rzekomo 
jest wytwarzana ludiom przez te zastrzyki.
'''

# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Wklej tekst i sprawdź jego wiarygodność!'),
    html.Hr(),

    html.Div([
        dcc.Textarea(
            id='input_text',
            value=example,
            style={'width': '100%', 'height': 200}
        ),
        html.Br(),
        # html.Button('Lematyzuj:', id='lem_button'),
        # html.Div(id='lem_text'),

        html.Button('Weryfikuj', id='ver_button'),
        html.Table([
            html.Tr([html.Td(['Klasyfikator NB:']), html.Td(id='NB')]),
            html.Tr([html.Td(['Klasyfikator SVC:']), html.Td(id='SVC')]),
            html.Tr([html.Td(['Klasyfikator RF:']), html.Td(id='RF')])
            
        ], style={'font-size': 26})
    ]),

])
# @app.callback(
#     Output(component_id='lem_text', component_property='children'),
#     Input(component_id='lem_button', component_property='n_clicks'),
#     State(component_id='input_text', component_property='value')
# )
# def lemmatize_text(n_clicks, text):
#     if n_clicks > 0:
#         text = strip_punctuation(text)
#         text = strip_non_alphanumeric(text)
#         text = replace_whitespace(text)
#         text = delete_escape_chars(text)
#         text = str.lower(text)
#         text = str.split(text)
#         text = delete_stop_words(text)

#         morf = morfeusz2.Morfeusz()
#         lemm_words = []
#         for word in text:
#             _, _, interpretation = morf.analyse(word)[0]
#             lem_word = (list(interpretation)[1]).lower()
#             lem_word_stripped = lem_word.split(':', 1)[0]
#             lemm_words.append(lem_word_stripped)
#         return f'{lemm_words}'
#     else:
#         return ''

@app.callback(
    [
        Output(component_id='NB', component_property='children'),
        Output(component_id='SVC', component_property='children'),
        Output(component_id='RF', component_property='children')
    ],
    Input(component_id='ver_button', component_property='n_clicks'),
    State(component_id='input_text', component_property='value')
)
def verificate_text(n_clicks, text):
    if n_clicks > 0:
        df = pd.DataFrame([[text]], columns=['Text'])

        df = preprocess(df)
        df = lemmatise(df)

        # morf = morfeusz2.Morfeusz()
        # lemm_words = []
        # for word in text:
        #     _, _, interpretation = morf.analyse(word)[0]
        #     lem_word = (list(interpretation)[1]).lower()
        #     lem_word_stripped = lem_word.split(':', 1)[0]
        #     lemm_words.append(lem_word_stripped)
        
        tfidf_vectorizer = load('tfidf.joblib')
        tfidf_input = tfidf_vectorizer.transform(df['Text'])

        nb_classifier = load('nb_classifier.joblib')
        svc_classifier = load('svc_classifier.joblib') 
        rf_classifier = load('rf_classifier.joblib') 

        nb_result = str(nb_classifier.predict(tfidf_input)[0])
        svc_result = str(svc_classifier.predict(tfidf_input)[0])
        rf_result = str(rf_classifier.predict(tfidf_input)[0])

        return nb_result, svc_result, rf_result
    else:
        return '', '', ''
# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>